In [1]:
import pandas as pd
import pulp
import math

In [33]:
demand_df = pd.read_csv("data/demand.csv")
vehicles_df = pd.read_csv("min_cost_combined.csv")
carbon_limits_df = pd.read_csv("data/carbon_emissions.csv")
current_fleet = pd.read_csv("current_fleet.csv")

In [34]:
demand_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256 entries, 0 to 255
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Year         256 non-null    int64 
 1   Size         256 non-null    object
 2   Distance     256 non-null    object
 3   Demand (km)  256 non-null    int64 
dtypes: int64(2), object(2)
memory usage: 8.1+ KB


In [35]:
vehicles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256 entries, 0 to 255
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Allocation               256 non-null    object 
 1   Operating Year           256 non-null    int64  
 2   Size                     256 non-null    object 
 3   Distance_demand          256 non-null    object 
 4   Demand (km)              256 non-null    int64  
 5   ID                       256 non-null    object 
 6   Vehicle                  256 non-null    object 
 7   Available Year           256 non-null    int64  
 8   Cost ($)                 256 non-null    int64  
 9   Yearly range (km)        256 non-null    int64  
 10  Distance_vehicle         256 non-null    object 
 11  Fuel                     256 non-null    object 
 12  carbon_emissions_per_km  256 non-null    float64
 13  insurance_cost           256 non-null    float64
 14  maintenance_cost         2

In [36]:
current_fleet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Operating Year           0 non-null      object
 1   Size                     0 non-null      object
 2   Distance_demand          0 non-null      object
 3   Demand (km)              0 non-null      object
 4   ID                       0 non-null      object
 5   Vehicle                  0 non-null      object
 6   Available Year           0 non-null      object
 7   Cost ($)                 0 non-null      object
 8   Yearly range (km)        0 non-null      object
 9   Distance_vehicle         0 non-null      object
 10  Fuel                     0 non-null      object
 11  carbon_emissions_per_km  0 non-null      object
 12  insurance_cost           0 non-null      object
 13  maintenance_cost         0 non-null      object
 14  fuel_costs_per_km        0 non-null      object
 15  T

In [37]:
carbon_limits_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 2 columns):
 #   Column                  Non-Null Count  Dtype
---  ------                  --------------  -----
 0   Year                    16 non-null     int64
 1   Carbon emission CO2/kg  16 non-null     int64
dtypes: int64(2)
memory usage: 388.0 bytes


In [38]:
prob = pulp.LpProblem("Fleet_Optimization", pulp.LpMinimize)

In [39]:
vehicles_df

,Allocation,Operating Year,Size,Distance_demand,Demand (km),ID,Vehicle,Available Year,Cost ($),Yearly range (km),Distance_vehicle,Fuel,carbon_emissions_per_km,insurance_cost,maintenance_cost,fuel_costs_per_km,Total_Cost
0,Diesel_S1_2023,2023,S1,D1,869181,Diesel_S1_2023,Diesel,2023,85000,102000,D4,B20,0.679882,4250.00,850.00,0.272267,90100.272267
1,Diesel_S1_2023,2023,S1,D2,2597094,Diesel_S1_2023,Diesel,2023,85000,102000,D4,B20,0.679882,4250.00,850.00,0.272267,90100.272267
2,Diesel_S1_2023,2023,S1,D3,3292011,Diesel_S1_2023,Diesel,2023,85000,102000,D4,B20,0.679882,4250.00,850.00,0.272267,90100.272267
3,Diesel_S1_2023,2023,S1,D4,414315,Diesel_S1_2023,Diesel,2023,85000,102000,D4,B20,0.679882,4250.00,850.00,0.272267,90100.272267
4,Diesel_S2_2023,2023,S2,D1,995694,Diesel_S2_2023,Diesel,2023,104000,106000,D4,B20,0.682431,5200.00,1040.00,0.273287,110240.273287
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,Diesel_S3_2038,2038,S3,D4,306910,Diesel_S3_2038,Diesel,2038,184123,73000,D4,HVO,0.109268,9206.15,1841.23,0.345573,195170.725573
252,Diesel_S4_2038,2038,S4,D1,23437,Diesel_S4_2038,Diesel,2038,235677,118000,D4,HVO,0.107193,11783.85,2356.77,0.339011,249817.959011
253,Diesel_S4_2038,2038,S4,D2,1148555,Diesel_S4_2038,Diesel,2038,235677,118000,D4,HVO,0.107193,11783.85,2356.77,0.339011,249817.959011
254,Diesel_S4_2038,2038,S4,D3,161386,Diesel_S4_2038,Diesel,2038,235677,118000,D4,HVO,0.107193,11783.85,2356.77,0.339011,249817.959011


In [ ]:
fleet_results = {}

for year in range(2023, 2039):
    unique_demand_combination_for_year = demand_df.groupby(['Size', 'Distance'])
    carbon_limit = carbon_limits_df[carbon_limits_df['Year'] == year]['Carbon emission CO2/kg'].values[0]

    total_emissions = []
    for (size, distance), group in unique_demand_combination_for_year:
        row = demand_df[(demand_df['Size'] == size) & 
                   (demand_df['Distance'] == distance) & 
                   (demand_df['Year'] == year)]
        
        demand = row['Demand (km)'].values[0] if not row.empty else None
        available_vehicles = []
        row = vehicles_df[(vehicles_df['Available Year'] == year) & 
                        (vehicles_df['Distance_demand'] == distance) & 
                        (vehicles_df['Size'] == size)]
        if not row.empty:
            available_vehicles.append(row['ID'].values[0])

        if len(current_fleet) > 0:
            pass
        
            # row =  current_fleet[(current_fleet['Available Year'] == year) & 
            #             (current_fleet['Distance_demand'] == distance) & 
            #             (current_fleet['Size'] == size)]
            # if not row.empty:
            #     available_vehicles.append(row['ID'].values[0])
        
        prob = pulp.LpProblem(f"Fleet_Optimization_{year}_{size}_{distance}", pulp.LpMinimize)
        
        for id in available_vehicles:
            row = vehicles_df[(vehicles_df['ID'] == id)]
            yearly_range = row['Yearly range (km)'].values[0] if not row.empty else None
            ub = math.ceil(demand/yearly_range)
            vehicle_vars = {id: pulp.LpVariable(f"num_vehicles_{id}", lowBound=0, cat='Integer')}            
                
        # Continuing the optimization logic
        # print(size, distance)
        # upBound=ub
        # print(ub)

        # Objective: Minimize total cost
        for id in available_vehicles:
            prob += pulp.lpSum(vehicle_vars[id] * (row['Cost ($)'].values[0] + 
                                              row['insurance_cost'].values[0] + 
                                              row['maintenance_cost'].values[0] + 
                                              row['fuel_costs_per_km'].values[0] * 
                                              row['Yearly range (km)'].values[0]))
            
            # Constraint: Meet the demand
            prob += pulp.lpSum(vehicle_vars[id] * row['Yearly range (km)'].values[0]) >= demand
            
        # print(prob)
        # Solve the optimization problem
        prob.solve()
        # print(vehicle_vars[id].varValue)
        if vehicle_vars[id].varValue > 0:
            new_row = {
                'Operating Year': year,
                'Size': size,
                'Distance': distance,
                'Demand (km)': demand,
                'ID': id,
                'Num_Vehicles': vehicle_vars[id].varValue,
                'Type': 'Buy' if year == 2023 else 'Use',
                'Fuel': row['Fuel'].values[0],
                'Available Year': row['Available Year'].values[0],
                'Cost ($)': row['Cost ($)'].values[0],
                'Yearly range (km)': row['Yearly range (km)'].values[0],
                'Distance_vehicle': row['Distance_vehicle'].values[0],
                'carbon_emissions_per_km': row['carbon_emissions_per_km'].values[0],
                'insurance_cost': row['insurance_cost'].values[0],
                'maintenance_cost': row['maintenance_cost'].values[0],
                'fuel_costs_per_km': row['fuel_costs_per_km'].values[0],
                'Topsis_Score': row['Topsis_Score'].values[0],
                'Rank': row['Rank'].values[0]
            }
            
            current_fleet = pd.concat([current_fleet, pd.DataFrame([new_row])], ignore_index=True)
            current_fleet.to_csv(f"fleet_{year}.csv")
    fleet_results[year] = current_fleet
    current_fleet = pd.DataFrame(columns=current_fleet.columns)

print("Fleet optimization complete.")

C:\Users\yasht\AppData\Local\Temp\ipykernel_21656\1775280281.py:80: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  current_fleet = pd.concat([current_fleet, pd.DataFrame([new_row])], ignore_index=True)
C:\Users\yasht\AppData\Local\Temp\ipykernel_21656\1775280281.py:80: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  current_fleet = pd.concat([current_fleet, pd.DataFrame([new_row])], ignore_index=True)
C:\Users\yasht\AppData\Local\Temp\ipykernel_21656\1775280281.py:80: FutureWarning: The behavior of

Fleet optimization complete.


In [ ]:
for year in range(2023, 2039):
    df = pd.read_csv(f"fleet_{year}.csv")
    df['sum'] = df.apply(lambda row: 
        row['Num_Vehicles'] * (
            row['Cost ($)'] + 
            row['insurance_cost'] + 
            row['maintenance_cost'] + 
            (
                row['fuel_costs_per_km'] * (row['Demand (km)']/row['Num_Vehicles'])
            )
        ), axis=1)

    Total = df['sum'].sum()
    print(f"{year}: {Total}")

2023: 26920360.734159406
2024: 28149456.132261902
2025: 29515791.901964642
2026: 31077990.06990113
2027: 33005125.800862145
2028: 34761166.59735003
2029: 37132499.90350282
2030: 38422871.196147785
2031: 40160378.76773394
2032: 43029136.65642424
2033: 45161710.42116553
2034: 47627904.69285491
2035: 50245037.46266701
2036: 52755154.62428418
2037: 55213133.56435092
2038: 55492347.868583396
